In [1]:
import numpy as np
import matplotlib.pyplot as plt
import urllib
from matplotlib import image

from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import h5py as h5py
import pickle
import os, cv2
from PIL import Image
import math



%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Data

In [4]:
img_urls = np.loadtxt('faces.csv', dtype=np.str)

OSError: faces.csv not found.

In [ ]:
def load_images_from_urls(urls):
    for i in range(len(urls)):
        try:
            urllib.request.urlretrieve(urls[i], 'images/img' + str(i) +'.png' )
        except:
            continue

# Try Basic-Yolo-Keras implementation

In [2]:
anchors = np.fromfile('tiny-yolo-voc-keras/tiny-yolo_anchors.txt', sep=', ')
IMAGE_W = 416
CHANNELS = 3
LABELS = 80
GRID = 13
CELL_WIDTH = IMAGE_W / GRID
BOX = LABELS+5
ANCHORS = int(len(anchors) / 2)

In [3]:
x_input = Input((IMAGE_W, IMAGE_W, CHANNELS))

#1
x = Conv2D(kernel_size=(3,3), filters=16, strides=(1,1), padding='SAME', name='conv2d_1', use_bias=False)(x_input)
x = BatchNormalization(name='batch_normalization_1')(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#2
x = Conv2D(kernel_size=(3,3), filters=32, strides=(1,1), padding='SAME', name='conv2d_2', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_2')(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#3
x = Conv2D(kernel_size=(3,3), filters=64, strides=(1,1), padding='SAME', name='conv2d_3', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_3')(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#4
x = Conv2D(kernel_size=(3,3), filters=128, strides=(1,1), padding='SAME', name='conv2d_4', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_4')(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#5
x = Conv2D(kernel_size=(3,3), filters=256, strides=(1,1), padding='SAME', name='conv2d_5', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_5')(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#6
x = Conv2D(kernel_size=(3,3), filters=512, strides=(1,1), padding='SAME', name='conv2d_6', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_6')(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2), strides=(1,1), padding='SAME')(x)

#7
x = Conv2D(kernel_size=(3,3), filters=1024, strides=(1,1), padding='SAME', name='conv2d_7', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_7')(x)
x = LeakyReLU()(x)

#8
x = Conv2D(kernel_size=(3,3), filters=1024, strides=(1,1), padding='SAME', name='conv2d_8', use_bias=False)(x)
x = BatchNormalization(name='batch_normalization_8')(x)
x = LeakyReLU()(x)

x = Conv2D(kernel_size=(1,1), filters=125, strides=(1,1), padding='SAME', name='conv2d_9')(x)

model = Model(inputs=x_input, outputs=x)

In [4]:
model.load_weights('tiny-yolo-voc-keras/tiny-yolo-voc.h5', by_name=True)

In [6]:
image = plt.imread('street.jpg')
resized = cv2.resize(image, (IMAGE_W, IMAGE_W))
image_data = np.expand_dims(resized, axis=0)
image_data = image_data
model.predict(image_data)

array([[[[-9.32819366e+01,  1.73876480e+02, -4.42053223e+02, ...,
           1.85315466e+03, -2.27983761e+01,  1.11552332e+03],
         [-1.13066238e+02,  1.48657455e+01, -9.20101440e+02, ...,
           2.84849902e+03,  6.92343811e+02,  2.24860864e+03],
         [ 3.03096039e+02,  4.47278595e+02, -1.29788501e+03, ...,
           2.40257690e+03,  1.14031995e+03,  2.69218091e+03],
         ...,
         [-1.78642853e+02,  4.32805359e+02, -5.14701965e+02, ...,
           1.04188550e+03,  1.18671265e+03,  2.21693433e+03],
         [ 1.77022171e+02,  9.46437317e+02, -5.29688965e+02, ...,
           1.16546497e+03,  4.96951996e+02,  1.95342200e+03],
         [ 4.63095978e+02,  6.81086060e+02, -4.83205566e+02, ...,
           8.65824524e+02, -3.41580844e+00,  1.13167603e+03]],

        [[-1.74720856e+02,  2.07038376e+02, -6.57596375e+02, ...,
           2.33626465e+03,  2.01556747e+02,  8.08713135e+02],
         [ 7.44041138e+01,  3.70872711e+02, -1.14944543e+03, ...,
           3.71480859e

In [7]:
class Box:
    def __init__(self):
        c,x,y,w,h = [float() for i in range(5)]

In [8]:
def box_intersection(a, b):
    w = overlap(a.x, a.w, b.x, b.w);
    h = overlap(a.y, a.h, b.y, b.h);
    if w < 0 or h < 0: return 0;
    area = w * h;
    return area;

def box_union(a, b):
    i = box_intersection(a, b);
    u = a.w * a.h + b.w * b.h - i;
    return u;

def box_iou(a, b):
    return box_intersection(a, b) / box_union(a, b);

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [10]:
def parse_netout(netout):
    boxes_confidences = np.array([netout[:,:,i*BOX] for i in range(ANCHORS)])
    boxes_xy = np.array([netout[:,:,(i*BOX + 1) : (i*BOX + 3)] for i in range(ANCHORS)])
    boxes_wh = np.array([netout[:,:,(i*BOX + 3) : (i*BOX + 5)] for i in range(ANCHORS)])
    boxes_class_probs = np.array([netout[:,:,(i*BOX + 5) : (i*BOX + BOX)] for i in range(ANCHORS)])
    return boxes_confidences, boxes_xy, boxes_wh, boxes_class_probs

In [11]:
test_netout = np.random.rand(13,13,425)
boxes_confidences, boxes_xy, boxes_wh, boxes_class_probs = parse_netout(test_netout)
print("boxes_confidences: ", boxes_confidences.shape)
print("boxes_xy: ", boxes_xy.shape)
print("boxes_wh: ", boxes_wh.shape)
print("boxes_class_probs: ", boxes_class_probs.shape)

boxes_confidences:  (5, 13, 13)
boxes_xy:  (5, 13, 13, 2)
boxes_wh:  (5, 13, 13, 2)
boxes_class_probs:  (5, 13, 13, 80)


In [12]:
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # initialize the list of picked indexes	
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
 
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")

In [13]:
def netout_to_boxes(netout, anchors):
    boxes_confidences, boxes_xy, boxes_wh, boxes_class_probs = parse_netout(netout)
    boxes = []

    for row in range(GRID):
        for col in range(GRID):
            for b in range(ANCHORS):
                box = Box()
                box.c = boxes_confidences[b, row, col]
                box.x = sigmoid(boxes_xy[b, row, col, 0]) + CELL_WIDTH * col
                box.y = sigmoid(boxes_xy[b, row, col, 1]) + CELL_WIDTH * row
                box.w = anchors[b*2] * np.exp(boxes_wh[b, row, col, 0])
                box.h = anchors[b*2 + 1] * np.exp(boxes_wh[b, row, col, 1])
                if box.w == float('inf'):
                    box.w = 90000
                if box.h == float('inf'):
                    box.h = 90000 
#                 print('x ', box.x)
#                 print('y ', box.y)
#                 print('w ', box.w)
#                 print('h ', box.h)
                
                boxes.append(box)
    return boxes

In [14]:
def frame_func(image):
    crop = image[300:650,500:,:]
    resized = cv2.resize(image, (IMAGE_W, IMAGE_W)) / 255.
    batch = np.expand_dims(resized, axis=0)
    out = model.predict(batch)
    boxes = netout_to_boxes(out[0], anchors)
    return draw_box(boxes,resized,[[0,IMAGE_W],[0,IMAGE_W]])

In [15]:
def draw_box(boxes,im,crop_dim):
    imgcv = im
    [xmin,xmax] = crop_dim[0]
    [ymin,ymax] = crop_dim[1]
    for b in boxes:
        h, w, _ = imgcv.shape
        left  = int ((b.x - b.w/2.) * w)
        right = int ((b.x + b.w/2.) * w)
        top   = int ((b.y - b.h/2.) * h)
        bot   = int ((b.y + b.h/2.) * h)
#         left = int(left*(xmax-xmin)/w + xmin)
#         right = int(right*(xmax-xmin)/w + xmin)
#         top = int(top*(ymax-ymin)/h + ymin)
#         bot = int(bot*(ymax-ymin)/h + ymin)

        if left  < 0    :  left = 0
        if right > w - 1: right = w - 1
        if top   < 0    :   top = 0
        if bot   > h - 1:   bot = h - 1
        thick = int((h + w) // 150)
        cv2.rectangle(imgcv, (left, top), (right, bot), (255,0,0), thick)

    return imgcv

In [16]:
image = plt.imread('street.jpg')
plt.imshow(frame_func(image))

IndexError: index 170 is out of bounds for axis 2 with size 125

In [70]:
anchors

array([0.57273 , 0.677385, 1.87446 , 2.06253 , 3.33843 , 5.47434 ,
       7.88282 , 3.52778 , 9.77052 , 9.16828 ])